In [2]:
from __future__ import print_function
from __future__ import division

import os
import sys
import argparse
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms

from utils.imagenet_dataset import ImageNetDataset
from torch.utils.data import DataLoader
from torchvision.transforms import ToPILImage
import models.inception_v3_imagenet as inception_v3
import models.generators as generators
import models.resnet_cifar10 as resnet
from argparse import Namespace

In [3]:
sys.argv = ['uniform_testing.ipynb', 
            '--no-cuda', 'False',
            '--seed', '5', 
            '--model-path', './checkpoints',
            '--test-batch-size', '50',
            '--target-count', '20',
            '--target-type', '0',
            '--target-label', '-1'
            ]
parser = argparse.ArgumentParser(description='Black-box Adversarial Attack')
parser.add_argument('--no-cuda', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=0, metavar='S',
                    help='random seed (default: 0)')
parser.add_argument('--model-path', default='../../checkpoints',
                    help='directory of model for saving checkpoint')
parser.add_argument('--test-batch-size', type=int, default=50, metavar='N',
                    help='input batch size for testing (default: 50)')
parser.add_argument('--target-count', type=int, default=20, metavar='N',
                    help='the amount of targets(default: 20)')
parser.add_argument('--target-type', type=int, default=0, metavar='N',
                    help='the method of choosing target label.\n0: ini_label + 1;\n1: all the other labels')
parser.add_argument('--target-label', type=int, default=-1, metavar='N',
                    help='target label, default: -1, which is (ini_label + 1) % 10')
args = parser.parse_args()
print(args)

Namespace(no_cuda='False', seed=5, model_path='./checkpoints', test_batch_size=50, target_count=20, target_type=0, target_label=-1)


In [4]:
# specify optimization related parameters
LR = 0.05  # learning rate
EPOCHS = 20  # total optimization epochs
NB_SAMPLE = 1000  # number of samples for adjusting lambda
MINI_BATCH = NB_SAMPLE // args.test_batch_size  # number of batches
INIT_COST = 1e-3  # initial weight of lambda

ATTACK_SUCC_THRESHOLD = 0.99  # attack success threshold
PATIENCE = 5  # patience for adjusting lambda, number of mini batches
COST_MULTIPLIER = 2  # multiplier for auto-control of weight (COST)
COST_MULTIPLIER_UP = COST_MULTIPLIER
COST_MULTIPLIER_DOWN = 10 ** 0.5  # changed from 2**1.5 to 10**0.5

EARLY_STOP_THRESHOLD = 1.0  # loss threshold for early stop
EARLY_STOP_PATIENCE = 5 * PATIENCE  # patience for early stop
EPSILON = 1e-8

In [10]:
def data_factory(dataset_type = 'ImageNet', no_cuda = False, batch_size = 1):
    use_cuda = not no_cuda and torch.cuda.is_available()
    if dataset_type == 'Cifar10':
        kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
        dataset = torchvision.datasets.CIFAR10(
            root='./datasets', train=False, download=True, 
            transform=transforms.Compose([
                transforms.CenterCrop(31),
                transforms.ToTensor()
            ])
        )
        return DataLoader(dataset, batch_size=batch_size, shuffle=False, **kwargs)
    elif dataset_type == 'ImageNet':
        im_size = 299
        mean_arr = (0.5, 0.5, 0.5)
        stddev_arr = (0.5, 0.5, 0.5)
        dataset = ImageNetDataset(
            image_dir='./datasets/imagenet1000',
            label_filepath="./datasets/imagenet_label.txt",
            transform=transforms.Compose([
                transforms.CenterCrop(im_size),
                transforms.ToTensor(),
                transforms.Normalize(mean_arr, stddev_arr)
            ]),
        )
        return DataLoader(dataset, batch_size=batch_size, shuffle=False)
 

def test_attack_success_rate(config, target_model, attack_algorithm, **kwargs):
    # sourcery skip: last-if-guard, use-fstring-for-concatenation
    """This is the unified model for testing attack success rate for adversarial success under a certain configuration.

    Args:
        config (Namespace): include the attack type(black or white, targeted or untargeted) and other necessary information
        target_model: the model to be attacked
        data_loader: batch_size = 1
    """
    print(f"=====Running test on {config.dataset_type} dataset, attacking model {config.target_model}.=====")
    dataloader = data_factory(config.dataset_type)
    if config.target_type == 'Untargeted':
        result = attack_algorithm(target_model, dataloader, config, **kwargs)  # untargeted attack

In [6]:
def attack_factory(attack_algorithm):
    mapping = {'B3D_w': B3D_attack_black,
               'greedyfool_w': greedyfool_attack_white, 
               'cornersearch_w': cornersearch_attack_black, 
               'PGD_attack_w': PGD_attack_white, 
               'homotopy_w': homotopy_attack_white, 
               'perturbation_w': perturbation_attack_black}
    return mapping[attack_algorithm]

def greedyfool_attack_white():
    pass

def greedyfool_attack_black():
    pass

def B3D_attack_black():
    pass

def cornersearch_attack_black():
    pass

def PGD_attack_white():
    pass

def homotopy_attack_white():
    pass

def perturbation_attack_black():
    pass

In [7]:
def greedyfool_attack_white(target_model, test_loader, config, generator):
    def clip(adv_A,real_A,eps):
        g_x=real_A-adv_A
        clip_gx=torch.clamp(g_x, min=-eps, max=eps)
        adv_x=real_A-clip_gx
        return adv_x
    
    def CWLoss(logits, target, kappa=0, tar=True if config.target_type == 'Targeted' else False):
        if config.dataset_type == 'ImageNet':
            target = torch.ones(logits.size(0)).type(torch.cuda.FloatTensor).mul(target.float())
            target_one_hot = Variable(torch.eye(1000).type(torch.cuda.FloatTensor)[target.long()].cuda())
            
            real = torch.sum(target_one_hot*logits, 1)
            other = torch.max((1-target_one_hot)*logits - (target_one_hot*10000), 1)[0]
            kappa = torch.zeros_like(other).fill_(kappa)
        elif config.dataset_type == 'Cifar10':
            target = torch.ones(logits.size(0)).type(torch.cuda.FloatTensor).mul(target.float())
            target_one_hot = Variable(torch.eye(10).type(torch.cuda.FloatTensor)[target.long()].cuda())
            
            real = torch.sum(target_one_hot*logits, 1)
            other = torch.max((1-target_one_hot)*logits - (target_one_hot*10000), 1)[0]  ## 这里的10000是什么原理
            kappa = torch.zeros_like(other).fill_(kappa)
        
        if tar:
            return torch.sum(torch.max(other - real, kappa))
        else:
            return torch.sum(torch.max(real - other, kappa))
    
    class AverageMeter(object):
        """Computes and stores the average and current value"""
        def __init__(self):
            self.reset()

        def reset(self):
            self.val = 0
            self.avg = 0
            self.sum = 0
            self.count = 0

        def update(self, val, n=1):
            self.val = val
            self.sum += val * n
            self.count += n
            self.avg = self.sum / self.count
    
    pool_kernel = 3
    Avg_pool = nn.AvgPool2d(pool_kernel, stride=1, padding=int(pool_kernel/2))
    
    print("Iter {0}".format(config.iter))
    print("EPS {0}".format(config.max_epsilon))
    Iter = config.iter
    eps = config.max_epsilon * 2 / 255
    im_size = config.image_size
    root = config.saving_root
    
    Baccu = []
    for i in range(1):
        temp_accu = AverageMeter()
        Baccu.append(temp_accu)
        
    num_count = []
    time_count = []
    if config.max_epsilon >= 128:
        boost = False
    else:
        boost = True
    print ("Boost:{0}".format(boost))
    for idx, data in enumerate(test_loader):
        if config.dataset_type == 'ImageNet':
            input_A, label_A, image_names = data
        elif config.dataset_type == 'Cifar10':
            input_A, label_A = data
            image_names = idx
        generator.eval()
        input_A = input_A.cuda()
        real_A = Variable(input_A, requires_grad=False)
        
        print(real_A.size())
        image_hill = generator(real_A * 0.5 + 0.5) * 0.5 + 0.5
        print(image_hill.size())
        pre_hill = 1 - image_hill
        pre_hill = pre_hill.view(1, 1, -1)
       
        np_hill = pre_hill.detach().cpu().numpy()
        percen = np.percentile(np_hill, 30)
        pre_hill = torch.max(pre_hill - percen, torch.zeros(pre_hill.size()).cuda())
        np_hill = pre_hill.detach().cpu().numpy()
        percen = np.percentile(np_hill, 75)
        pre_hill /= percen
        pre_hill = torch.clamp(pre_hill, 0, 1)
        print(pre_hill.size())
        pre_hill = Avg_pool(pre_hill) 
        SIZE = int(im_size * im_size)
        
        loss_adv = CWLoss
        
        logist_B = target_model(real_A)
        _, target = torch.max(logist_B, 1)
        adv = real_A
        ini_num = 1
        grad_num = ini_num
        mask = torch.zeros(1, 3, SIZE).cuda()

        temp_eps = eps / 2

        ##### Increasing
        for iters in range(Iter):
            # print (iters)
            temp_A = Variable(adv.data, requires_grad=True)
            logist_B = target_model(temp_A)
            _, pre = torch.max(logist_B, 1)
            
            if target.cpu().data.float() != pre.cpu().data.float():
                break
            Loss = loss_adv(logist_B, target, -100, False) / real_A.size(0)
            
            target_model.zero_grad()
            if temp_A.grad is not None:
                temp_A.grad.data.fill_(0)
            Loss.backward()
            
            grad = temp_A.grad
            abs_grad = torch.abs(grad).view(1, 3, -1).mean(1, keepdim=True)
            print(pre_hill.size())
            abs_grad = abs_grad * pre_hill
            if not boost:
                abs_grad = abs_grad * (1 - mask)
            _, grad_sort_idx = torch.sort(abs_grad)
            grad_sort_idx = grad_sort_idx.view(-1)
            grad_idx = grad_sort_idx[-grad_num:]
            mask[0, :, grad_idx] = 1.
            temp_mask = mask.view(1, 3, im_size, im_size)
            grad = temp_mask * grad
            
            abs_grad = torch.abs(grad)
            abs_grad = abs_grad / torch.max(abs_grad)
            normalized_grad = abs_grad * grad.sign()
            scaled_grad = normalized_grad.mul(temp_eps)
            temp_A = temp_A - scaled_grad
            temp_A = clip(temp_A, real_A, eps)
            adv = torch.clamp(temp_A, -1, 1)
            if boost:
                grad_num += ini_num
            
        final_adv = adv
        adv_noise = real_A - final_adv
        adv = final_adv
        
        abs_noise = torch.abs(adv_noise).view(1, 3, -1).mean(1, keepdim=True)
        temp_mask = abs_noise != 0
        modi_num = torch.sum(temp_mask).data.clone().item()
        
        reduce_num = modi_num
        reduce_count = 0
        ###### Reducing
        if modi_num > 2:
            reduce_idx = 0
            while reduce_idx < reduce_num and reduce_count < 3000:
                reduce_count += 1
                adv_noise = real_A - adv
                
                abs_noise = torch.abs(adv_noise).view(1, 3, -1).mean(1, keepdim=True)
                reduce_mask = abs_noise != 0
                reduce_mask = reduce_mask.repeat(1, 3, 1).float()
                abs_noise[abs_noise == 0] = 3.
                
                reduce_num = torch.sum(reduce_mask).data.clone().item() / 3
                if reduce_num == 1:
                    break
                
                noise_show, noise_sort_idx = torch.sort(abs_noise)
                noise_sort_idx = noise_sort_idx.view( -1)
                
                noise_idx = noise_sort_idx[reduce_idx]
                reduce_mask[0,:,noise_idx] = 0.
                temp_mask = reduce_mask.view(1,3,int(im_size),int(im_size))
                noise = temp_mask * adv_noise
                
                abs_noise = torch.abs(noise)
                abs_noise = abs_noise / torch.max(abs_noise)
                normalized_grad = abs_noise * noise.sign()
                
                with torch.no_grad():
                    target_model.eval()
                    step = int(max(int(config.max_epsilon/10.),1))
                    a = [i for i in range(0, int(config.max_epsilon+step), step)]
                    search_num = len(a)
                    a = np.asarray(a)*2/255. 
                    ex_temp_eps = torch.from_numpy(a).view(-1,1,1,1).float().cuda()
                    ex_normalized_grad = normalized_grad.repeat(int(search_num),1,1,1)
                    ex_scaled_grad = ex_normalized_grad.mul(ex_temp_eps)
                    ex_real_A = real_A.repeat(int(search_num),1,1,1)
                    ex_temp_A = ex_real_A - ex_scaled_grad
                    ex_temp_A = clip(ex_temp_A, ex_real_A, eps)
                    ex_adv = torch.clamp(ex_temp_A, -1, 1)
                    ex_temp_A = Variable(ex_adv.data, requires_grad=False)
                    ex_logist_B = target_model(ex_temp_A)
                    _,pre=torch.max(ex_logist_B,1)
                    comp = torch.eq(target.cpu().data.float(), pre.cpu().data.float())
                    top1 = torch.sum(comp).float() / pre.size(0)
                    if top1 != 1: ##### exists at least one adversarial sample
                        found = False
                        for i in range(int(search_num)):
                            if comp[i] == 0:
                                temp_adv = ex_temp_A[i:i+1]
                                logist_B = target_model(temp_adv)
                                _,pre=torch.max(logist_B,1)
                                new_comp = torch.eq(target.cpu().data.float(), pre.cpu().data.float())
                                if torch.sum(new_comp) != 0:
                                    continue
                                found = True
                                adv = temp_adv
                                break
                        if found == False:
                            reduce_idx += 1
                    else:
                        reduce_idx += 1
                        
        
        adv_noise = real_A - adv
        abs_noise = torch.abs(adv_noise).view(1, 3, -1).mean(1, keepdim=True)
        temp_mask = abs_noise != 0
        
        reduce_num = torch.sum(temp_mask).data.clone().item()
        L1_X_show = torch.max(torch.abs(real_A - adv)) * 255. / 2

        num_count.append(reduce_num)

        logist_B = target_model(adv)
        _, pre = torch.max(logist_B, 1)
        top1 = torch.sum(torch.eq(target.cpu().data.float(), pre.cpu().data.float()).float()) / input_A.size(0)

        top1 = torch.from_numpy(np.asarray([(1 - top1)*100])).float().cuda()
        Baccu[0].update(top1[0], input_A.size(0))
        
        print('[{it:.2f}][{name}] '
                      'BTOP1: {BTOP1.avg:.2f} '
                      'lossX: {ori:d}/{redu:d} '
                      'L1: {l1:.1f} '
                      'M&m {mean:.2f}/{median:.2f} '
                      'Num: {num}'.format(
                          it = float(idx*100)/len(test_loader),
                          name = image_names[0].split('_')[-1],
                          BTOP1 = Baccu[0],
                          ori = int(modi_num),
                          redu = int(reduce_num),
                          l1 = L1_X_show.data.clone().item(),
                          num = grad_num,
                          mean = np.mean(num_count),
                          median = np.median(num_count)))

       
        if not os.path.exists(root):
            os.makedirs(root)
        if not os.path.exists(os.path.join(root,'clean')):
            os.makedirs(os.path.join(root,'clean'))
        if not os.path.exists(os.path.join(root,'adv')):
            os.makedirs(os.path.join(root,'adv'))
        if not os.path.exists(os.path.join(root,'show')):
            os.makedirs(os.path.join(root,'show'))

        hill_imgs = pre_hill.view(pre_hill.size(0),1,im_size,im_size).repeat(1,3,1,1)
        
        if modi_num >= 0.:
            for i in range(input_A.size(0)):
                clip_img = ToPILImage()((adv[i].data.cpu()+ 1) / 2) 
                real_img = ToPILImage()((real_A[i].data.cpu()+ 1) / 2)
                adv_path = os.path.join(root,'adv' ,image_names[i] +'_' +str(int(modi_num))+'.png')
                clip_img.save(adv_path)
                real_path = os.path.join(root,'clean', image_names[i] +'_' +str(int(modi_num))+'.png')
                real_img.save(real_path)
                
                if True:
                    hill_img = ToPILImage()(hill_imgs[i].data.cpu())
                    temp_adv = torch.abs(adv_noise[i].data.cpu())
                    temp_adv = temp_adv / torch.max(temp_adv)
                    temp_adv = 1 - temp_adv
                    adv_img = ToPILImage()(temp_adv)

                    temp_hill = image_hill[i].data.cpu()
                    
                    temp_hill = 1 - temp_hill
                    temp_hill = temp_hill.view(1,im_size,im_size).repeat(3,1,1)
                    
                    temp_hill = ToPILImage()(temp_hill)
                    final = Image.fromarray(np.concatenate([temp_hill, hill_img, real_img,adv_img,clip_img],1))
                    final.save( os.path.join(root,'show', image_names[i] +'_' +str(int(modi_num))+'.png'))


In [10]:
def homotopy_attack_white(target_model, test_loader, config):
    def CWLoss(logits, target, kappa=0, tar = True):
        target = torch.ones(logits.size(0)).type(torch.cuda.FloatTensor).mul(target.float())
        target_one_hot = Variable(torch.eye(1000).type(torch.cuda.FloatTensor)[target.long()].cuda())
        
        real = torch.sum(target_one_hot*logits, 1)
        other = torch.max((1-target_one_hot)*logits - (target_one_hot*10000), 1)[0]
        kappa = torch.zeros_like(other).fill_(kappa)
        
        if tar:
            return torch.sum(torch.max(other-real, kappa))
        else :
            return torch.sum(torch.max(real-other, kappa))
    
    def after_attack(x, net, original_img, original_class, target_class, post, loss_type, tar, iters, val_w1, val_w2, max_epsilon):

        if post == 1:
            s1 = 1e-3
            s2 = 1e-4
            max_iter = 40000
        else:
            s1 = val_w2
            s2 = val_w1
            max_iter = iters

        mask = torch.where(torch.abs(x.data) > 0, torch.ones(1).cuda(), torch.zeros(1).cuda())

        logist = net(x.data+original_img.data)
        _,target=torch.max(logist,1)


        pre_x = x.data

        for i in range(max_iter):

            temp = Variable(x.data, requires_grad=True)
            logist = net(temp + original_img.data)
            if tar == 1:
                if loss_type == 'ce':
                    ce = torch.nn.CrossEntropyLoss()
                    Loss = ce(logist,torch.ones(1).long().cuda()*target_class)
                elif loss_type == 'cw':
                    Loss = CWLoss(logist, torch.ones(1).long().cuda()*target_class, kappa=0, tar = True)
            else:
                Loss = CWLoss(logist, torch.ones(1).long().cuda()*target_class, kappa=0, tar = False)

            net.zero_grad()
            if temp.grad is not None:
                temp.grad.data.fill_(0)
            Loss.backward()
            grad = temp.grad

        
            temp2 = Variable(x.data, requires_grad=True)
            Loss2 = torch.norm(temp2, p=float("inf"))
            net.zero_grad()
            if temp2.grad is not None:
                temp2.grad.data.fill_(0)
            Loss2.backward()
            grad2 = temp2.grad


            pre_x = x.data

            pre_noise = temp2.data
            if post == 0:
                temp2 = temp2.data - s1*grad2.data*mask - s2*grad.data*mask
            else:
                temp2 = temp2.data - s1*grad2.data*mask

            thres = max_epsilon
            temp2 = torch.clamp(temp2.data, -thres, thres)
            temp2 = torch.clamp(original_img.data+temp2.data, 0, 1)
        
            x = temp2.data - original_img.data
        

            logist = net(x.data + original_img.data)
            _,pre=torch.max(logist,1)
            if(post == 1):
                if tar ==  1:
                    if(pre.item() != target_class):
                        success = 1
                        return pre_x
                        break
                else:
                    if(pre.item() == target_class):
                        success = 1
                        return pre_x
                        break

        return x

    def F(x, loss_type, net, lambda1, original_img, target_class, tar):
        temp = Variable(x.data, requires_grad=False)
        logist = net(temp+original_img.data)
        if tar == 1:
            if loss_type == 'ce':
                ce = torch.nn.CrossEntropyLoss()
                Loss = ce(logist,torch.ones(1).long().cuda()*target_class)
            elif loss_type == 'cw':
                Loss = CWLoss(logist, torch.ones(1).long().cuda()*target_class, kappa=0, tar = True)
        else:
            Loss = CWLoss(logist, torch.ones(1).long().cuda()*target_class, kappa=0, tar = False)
        res = Loss.item() + lambda1*torch.norm(x.data,0).item()
        net.zero_grad()
        return res

    def prox_pixel(x, alpha, lambda1, original_img, max_epsilon):

        temp_x = x.data * torch.ones(x.shape).cuda()

        thres = max_epsilon
        clamp_x = torch.clamp(temp_x.data, -thres, thres)

        temp_img = original_img.data + clamp_x.data
        temp_img = torch.clamp(temp_img.data, 0, 1)
        clamp_x = temp_img.data - original_img.data

        val = 1 / (2*alpha*lambda1)
        cond = 1 + val * (clamp_x-temp_x)*(clamp_x-temp_x) > val * temp_x*temp_x
        cond = cond.cuda()

        res = torch.zeros(x.shape).cuda()
        res = torch.where(cond, res, clamp_x.data)
        return res

    def nmAPG(x0, loss_type, net, eta, delta, rho, original_img, lambda1, search_lambda_inc, search_lambda_dec, target_class, original_class, tar, max_update, maxiter, max_epsilon):

        x0_norm0 = torch.norm(torch.ones(x0.shape).cuda()*x0.data,0).item()
        max_update = max_update

        temp = Variable(x0.data, requires_grad=False)
        logist = net(temp+original_img.data)
        if tar == 1:
            if loss_type == 'ce':
                ce = torch.nn.CrossEntropyLoss()
                Loss = ce(logist, torch.ones(1).long().cuda()*target_class)
            elif loss_type == 'cw':
                Loss = CWLoss(logist, torch.ones(1).long().cuda()*target_class, kappa=0, tar=True)
        else:
            Loss = CWLoss(logist, torch.ones(1).long().cuda()*target_class, kappa=0, tar=False)
        net.zero_grad()

        z = x0
        y_pre = torch.zeros(original_img.shape).cuda()

        pre_loss = 0
        cur_loss = 0

        counter = 0
        success = 0

        alpha_y = 1e-3
        alpha_x = 1e-3

        alpha_min = 1e-20
        alpha_max = 1e20
        x_pre = x0
        x = x0
        t = 1
        t_pre = 0
        c = Loss + lambda1*torch.norm(x.data,0)
        q = 1
        k = 0
        while True:
            y = x + t_pre/t*(z-x) + (t_pre-1)/t*(x-x_pre)

            if k > 0:
                s = y - y_pre.data

                #gradient of yk
                temp_y = Variable(y.data, requires_grad=True)
                logist_y = net(temp_y+original_img.data)
                if tar == 1:
                    if loss_type == 'ce':
                        ce = torch.nn.CrossEntropyLoss()
                        Loss_y = ce(logist_y, torch.ones(1).long().cuda()*target_class)
                    elif loss_type == 'cw':
                        Loss_y = CWLoss(logist_y, torch.ones(1).long().cuda()*target_class, kappa=0, tar = True)
                else:
                    Loss_y = CWLoss(logist_y, torch.ones(1).long().cuda()*target_class, kappa=0, tar = False)
                net.zero_grad()
                if temp_y.grad is not None:
                    temp_y.grad.data.fill_(0)
                Loss_y.backward() 
                grad_y = temp_y.grad

                #gradient of yk-1
                temp_y_pre = Variable(y_pre.data, requires_grad=True)
                logist_y_pre = net(temp_y_pre+original_img.data)
                if tar == 1:
                    if loss_type == 'ce':
                        ce = torch.nn.CrossEntropyLoss()
                        Loss_y_pre = ce(logist_y_pre, torch.ones(1).long().cuda()*target_class)
                    elif loss_type == 'cw':
                        Loss_y_pre = CWLoss(logist_y_pre, torch.ones(1).long().cuda()*target_class, kappa=0, tar = True)
                else:
                    Loss_y_pre = CWLoss(logist_y_pre, torch.ones(1).long().cuda()*target_class, kappa=0, tar = False)
                net.zero_grad()
                if temp_y_pre.grad is not None:
                    temp_y_pre.grad.data.fill_(0)
                Loss_y_pre.backward() 
                grad_y_pre = temp_y_pre.grad

                r = grad_y - grad_y_pre
                
                #prevent error caused by numerical inaccuracy
                if torch.norm(s,1) < 1e-5:
                    s = torch.ones(1).cuda()*1e-5
                
                if torch.norm(r,1) < 1e-10:
                    r = torch.ones(1).cuda()*1e-10

                alpha_y = torch.sum(s*r)/torch.sum(r*r)
                alpha_y = alpha_y.item()
            
            temp_alpha = alpha_y

            if temp_alpha < alpha_min:
                temp_alpha = alpha_min

            if temp_alpha > alpha_max:
                temp_alpha = alpha_max

            if np.isnan(temp_alpha):
                temp_alpha = alpha_min
            alpha_y = temp_alpha

            count1 = 0
            while True:
                count1 = count1 + 1
                if count1 > 1000:
                    break

                temp_y = Variable(y.data, requires_grad=True)
                logist_y = net(temp_y+original_img.data)
                if tar == 1:
                    if loss_type == 'ce':
                        ce = torch.nn.CrossEntropyLoss()
                        Loss_y = ce(logist_y,torch.ones(1).long().cuda()*target_class)
                    elif loss_type == 'cw':
                        Loss_y = CWLoss(logist_y, torch.ones(1).long().cuda()*target_class, kappa=0, tar = True)
                else:
                    Loss_y = CWLoss(logist_y, torch.ones(1).long().cuda()*target_class, kappa=0, tar = False)
                net.zero_grad()
                if temp_y.grad is not None:
                    temp_y.grad.data.fill_(0)
                Loss_y.backward() 
                grad_y = temp_y.grad

                z = prox_pixel(x=y-alpha_y*grad_y,alpha=alpha_y,lambda1=lambda1,original_img=original_img, max_epsilon=max_epsilon)
                
                #increase lambda
                if(search_lambda_inc == 1):
                    if(torch.norm(z,1) != 0):
                        return 0
                    else:
                        return 1

                #decrease lambda
                if(search_lambda_dec == 1):
                    if(torch.norm(z,1) == 0):
                        return 0
                    else:
                        return lambda1

                alpha_y = alpha_y * rho
                cond1 = F(z, loss_type, net, lambda1, original_img,target_class,tar) <= F(y, loss_type, net, lambda1, original_img,target_class,tar) - delta*(torch.norm(z-y,2)*torch.norm(z-y,2))
                cond2 = F(z, loss_type, net, lambda1, original_img,target_class,tar) <= c - delta*(torch.norm(z-y,2)*torch.norm(z-y,2))

                if(cond1 | cond2):
                    break
            
            if F(z, loss_type, net, lambda1, original_img,target_class,tar) <= c - delta*(torch.norm(z-y,2)*torch.norm(z-y,2)):
                x_pre = x
                temp_norm0 = torch.norm(torch.ones(z.shape).cuda()*z.data,0).item()
                if np.abs(temp_norm0 - x0_norm0) > max_update:
                    temp_z = torch.abs((torch.ones(z.shape).cuda()*z.data).reshape(1,-1))
                    val, idx = temp_z.topk(k=int(x0_norm0+max_update))

                    thres = val[0,int(x0_norm0+max_update-1)]
                    z = torch.where(torch.abs(z.data) < thres, torch.zeros(1).cuda(), z.data)
                    x = z
                else:
                    x = z
            else:

                if k > 0:
                    s = x - y_pre.data

                    temp_x = Variable(x.data, requires_grad=True)
                    logist_x = net(temp_x+original_img.data)
                    if tar == 1:
                        if loss_type == 'ce':
                            ce = torch.nn.CrossEntropyLoss()
                            Loss_x = ce(logist_x,torch.ones(1).long().cuda()*target_class)
                        elif loss_type == 'cw':
                            Loss_x = CWLoss(logist_x, torch.ones(1).long().cuda()*target_class, kappa=0, tar = True)
                    else:
                        Loss_x = CWLoss(logist_x, torch.ones(1).long().cuda()*target_class, kappa=0, tar = False)
                    net.zero_grad()
                    if temp_x.grad is not None:
                        temp_x.grad.data.fill_(0)
                    Loss_x.backward() 
                    grad_x = temp_x.grad

                    temp_y_pre = Variable(y_pre.data, requires_grad=True)
                    logist_y_pre = net(temp_y_pre+original_img.data)
                    if tar == 1:
                        if loss_type == 'ce':
                            ce = torch.nn.CrossEntropyLoss()
                            Loss_y_pre = ce(logist_y_pre,torch.ones(1).long().cuda()*target_class)
                        elif loss_type == 'cw':
                            Loss_y_pre = CWLoss(logist_y_pre, torch.ones(1).long().cuda()*target_class, kappa=0, tar = True)
                    else:
                        Loss_y_pre = CWLoss(logist_y_pre, torch.ones(1).long().cuda()*target_class, kappa=0, tar = False)
                    net.zero_grad()
                    if temp_y_pre.grad is not None:
                        temp_y_pre.grad.data.fill_(0)
                    Loss_y_pre.backward() 
                    grad_y_pre = temp_y_pre.grad
                
                    r = grad_x - grad_y_pre

                    if torch.norm(s, 1) < 1e-5:
                        s = torch.ones(1).cuda() * 1e-5

                    if torch.norm(r,1) < 1e-10:
                        r = torch.ones(1).cuda()*1e-10
                    
                    alpha_x = torch.sum(s*r)/torch.sum(r*r)
                    alpha_x = alpha_x.item()

                temp_alpha = alpha_x

            
                if temp_alpha < alpha_min:
                    temp_alpha = alpha_min

                if temp_alpha > alpha_max:
                    temp_alpha = alpha_max
                if np.isnan(temp_alpha):
                    temp_alpha = alpha_min
                alpha_x = temp_alpha

                count2 = 0
                while True:
                    count2 = count2 + 1

                    if count2 > 10:
                        break

                    temp_x = Variable(x.data, requires_grad=True)
                    logist_x = net(temp_x + original_img.data)
                    if tar == 1:
                        if loss_type == 'ce':
                            ce = torch.nn.CrossEntropyLoss()
                            Loss_x = ce(logist_x, torch.ones(1).long().cuda() * target_class)
                        elif loss_type == 'cw':
                            Loss_x = CWLoss(logist_x, torch.ones(1).long().cuda() * target_class, kappa=0, tar=True)
                    else:
                        Loss_x = CWLoss(logist_x, torch.ones(1).long().cuda() * target_class, kappa=0, tar=False)
                    net.zero_grad()
                    if temp_x.grad is not None:
                        temp_x.grad.data.fill_(0)
                    Loss_x.backward()
                    grad_x = temp_x.grad

                    v = prox_pixel(x=x-alpha_x*grad_x,alpha=alpha_x,lambda1=lambda1,original_img=original_img, max_epsilon=max_epsilon)
                    alpha_x = rho * alpha_x
                    cond3 = F(v, loss_type, net, lambda1, original_img,target_class,tar) <= c - delta*(torch.norm(v-x,2)*torch.norm(v-x,2))

                    if cond3:
                        break
                    if torch.abs(F(v, loss_type, net, lambda1, original_img,target_class,tar) - (c - delta*(torch.norm(v-x,2)*torch.norm(v-x,2)) )) < 1e-3:
                        break

                
                if F(z, loss_type, net, lambda1, original_img,target_class,tar) <= F(v, loss_type, net, lambda1, original_img,target_class,tar):
                    x_pre = x
                    temp_norm0 = torch.norm(torch.ones(z.shape).cuda() * z.data, 0).item()
                    if np.abs(temp_norm0 - x0_norm0) > max_update:
                        temp_z = torch.abs((torch.ones(z.shape).cuda() * z.data).reshape(1, -1))
                        val, idx = temp_z.topk(k=int(x0_norm0 + max_update))

                        thres = val[0, int(x0_norm0 + max_update - 1)]
                        z = torch.where(torch.abs(z.data) < thres, torch.zeros(1).cuda(), z.data)
                        x = z
                    else:
                        x = z
                else:
                    x_pre = x
                    temp_norm0 = torch.norm(torch.ones(v.shape).cuda() * v.data, 0).item()
                    if np.abs(temp_norm0 - x0_norm0) > max_update:
                        temp_v = torch.abs((torch.ones(v.shape).cuda() * v.data).reshape(1, -1))
                        val, idx = temp_v.topk(k=int(x0_norm0 + max_update))
                        thres = val[0, int(x0_norm0 + max_update - 1)]
                        v = torch.where(torch.abs(v.data) < thres, torch.zeros(1).cuda(), v.data)
                        x = v
                    else:
                        x = v


            thres = max_epsilon
            x = torch.clamp(x.data,-thres,thres)
            temp_img = original_img.data + x.data
            temp_img = torch.clamp(temp_img.data, 0, 1)
            x = temp_img.data - original_img.data

            y_pre = y.data
            t = (np.sqrt(4*t*t+1)+1)/2
            q = eta*q + 1
            c = (eta*q*c + F(x, loss_type, net, lambda1, original_img,target_class,tar))/q

            logist = net(x.data+original_img.data)
            _,target=torch.max(logist,1)

            k = k + 1

            pre_loss = cur_loss

            if tar == 0:
                cur_loss = CWLoss(logist.data, torch.ones(1).long().cuda()*target_class, kappa=0, tar = False).item()
            else:
                if loss_type == 'cw':
                    cur_loss = CWLoss(logist.data, torch.ones(1).long().cuda()*target_class, kappa=0, tar = True).item()
                else:
                    ce = torch.nn.CrossEntropyLoss()
                    cur_loss = ce(logist.data, torch.ones(1).long().cuda() * target_class).item()
            net.zero_grad()

            #success
            if tar == 1:
                if (target == target_class):
                    success = 1
                    break
            else:
                if ((target != target_class)):
                    success = 1
                    break

            if ((success == 0) and (k >= maxiter) and (np.abs(pre_loss-cur_loss) < 1e-3) and (counter==1)):
                break

            if((k >= maxiter) and (np.abs(pre_loss-cur_loss) < 1e-3)):
                counter = 1

        return x, success

    def search_lambda(loss_type, net, original_img, target_class, original_class, tar, val_c, max_update, maxiter, max_epsilon):

        lambda1 = 1e-6
        x0 = torch.zeros(original_img.shape).cuda()
        k1 = 0
        while True:
            k1 = k1 + 1
            temp = nmAPG(x0=x0, loss_type=loss_type, net=net, eta=0.9, delta=0.3, rho=0.8, original_img=original_img, lambda1=lambda1,
                        search_lambda_inc=1, search_lambda_dec=0, target_class=target_class, original_class=original_class, tar=tar, max_update=max_update, maxiter=maxiter, max_epsilon=max_epsilon)
            if temp == 0:
                lambda1 = lambda1 + 1e-6
            if temp == 1:
                break

        k2 = 0
        while True:
            k2 = k2 + 1
            temp = nmAPG(x0=x0, loss_type=loss_type, net=net, eta=0.9, delta=0.3, rho=0.8, original_img=original_img, lambda1=lambda1,
                        search_lambda_inc=0, search_lambda_dec=1, target_class=target_class, original_class=original_class, tar=tar,max_update=max_update, maxiter=maxiter, max_epsilon=max_epsilon)
            if temp == 0:
                lambda1 = lambda1 * 0.99
            else:
                break

        lambda1 = lambda1 * val_c
        print('attack lambda = ', lambda1)

        return lambda1

    def homotopy(loss_type, net, original_img, target_class, original_class, tar, max_epsilon, dec_factor, val_c, val_w1, val_w2, max_update, maxiter, val_gamma):

        lambda1 = search_lambda(loss_type, net, original_img, target_class, original_class,tar, val_c, max_update, maxiter, max_epsilon=max_epsilon)

        x = torch.zeros(original_img.shape).cuda()
        pre_norm0 = 0
        cur_norm0 = 0

        max_norm0 = torch.norm(torch.ones(x.shape).cuda(),0).item()
        outer_iter = 0
        val_max_update = max_update
        while True:
            outer_iter = outer_iter + 1

            x, success = nmAPG(x0=x, loss_type=loss_type, net=net, eta=0.9, delta=0.3, rho=0.8, original_img=original_img, lambda1=lambda1,
                            search_lambda_inc=0, search_lambda_dec=0, target_class=target_class, original_class=original_class, tar=tar, max_update=max_update, maxiter=maxiter, max_epsilon=max_epsilon)
            max_update = val_max_update
            pre_norm0 = cur_norm0
            cur_norm0 = torch.norm(torch.ones(x.shape).cuda()*x.data,0).item()
            cur_norm1 = torch.norm(torch.ones(x.shape).cuda() * x.data, 1).item()

            #attack fail
            if(cur_norm0 > max_norm0*0.95 and outer_iter*max_update > max_norm0*0.95):
                break

            iters = 0
            if (cur_norm1 <= cur_norm0 * max_epsilon * val_gamma):
                max_update = 10
                iters = 200
                if cur_norm0 >= 500:
                    iters = 400
                if cur_norm0 >= 1000:
                    iters = 600
                if cur_norm0 >= 1500:
                    iters = 800
                if cur_norm0 >= 2000:
                    iters = 1000
                if cur_norm0 >= 2500:
                    iters = 1200

            if success == 0:
                x = after_attack(x, net, original_img, original_class, target_class, post=0, loss_type=loss_type, tar=tar, iters=iters, val_w1=val_w1, val_w2=val_w2, max_epsilon=max_epsilon)
                lambda1 = dec_factor * lambda1
            else:
                break

            logi = net(x.data+original_img.data)
            _,cur_class=torch.max(logi,1)
            if tar == 1:
                if((cur_class == target_class)):
                    break
            else:
                if((cur_class != target_class)):
                    break

        x = after_attack(x, net, original_img, original_class, target_class, post=1, loss_type=loss_type,tar=tar, iters=iters, val_w1=val_w1, val_w2=val_w2, max_epsilon=max_epsilon)

        return x



In [13]:
from argparse import Namespace


##### Target model loading
# netT = models.inception_v3(pretrained=True)
netT = inception_v3.inception_v3(pretrained=False) # 可以该做resnet18
netT.load_state_dict(torch.load('./checkpoints/inception_v3.pth'))
netT.eval()
netT.cuda()

##### Generator loading for distortion map
netG = generators.Res_ResnetGenerator(3, 1, 16, norm_type='batch', act_type='relu')
netG = torch.nn.DataParallel(netG, device_ids=range(torch.cuda.device_count()))
netG.load_state_dict(torch.load('./checkpoints/G_imagenet.pth'))
print(netG)
netG.cuda()
netG.eval()

def GanGenerator(dataset_type):
    netG = generators.Res_ResnetGenerator(3, 1, 16, norm_type='batch', act_type='relu', dataset_type=dataset_type)
    netG = torch.nn.DataParallel(netG, device_ids=range(torch.cuda.device_count()))
    netG.load_state_dict(torch.load('./checkpoints/G_imagenet.pth'))
    netG.cuda()
    netG.eval()
    return netG

def configuration():
    config = Namespace()
    config.target_type = 'Untargeted'
    config.dataset_type = 'Cifar10'
    config.target_model = 'resnet18'
    config.iter = 50
    config.max_epsilon = 100
    config.image_size = 32
    config.saving_root = './result/Greedyfool/untargeted/' + config.dataset_type + '/'
    return config


DataParallel(
  (module): Res_ResnetGenerator(
    (act): ReLU(inplace=True)
    (res1): ResnetBlock(
      (conv_block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): ReLU(inplace=True)
        (4): ReflectionPad2d((1, 1, 1, 1))
        (5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (res2): ResnetBlock(
      (conv_block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): ReLU(inplace=True)
        (4): ReflectionPad2d((1, 1, 1, 1))
        (5): Conv2d(64, 64, kernel_size=(3, 3), stride=

In [ ]:
##### configuration
config = configuration()

##### Target model loading
netT = target_net_factory(config.target_model)
attack_algorithm = attack_factory('greedyfool_w')

##### Generator loading for distortion map
netG = GanGenerator(config.dataset_type)
test_attack_success_rate(config, netT, attack_algorithm, generator=netG)